In [ ]:
import pandas as pd
import json

file_path = 'yelp_academic_dataset_business.json'
df_negocios = pd.read_json(file_path, lines=True)

df_negocios.head()


In [30]:
file_path = 'yelp_academic_dataset_tip.json'
df_tip = pd.read_json(file_path, lines=True)

df_tip.head()

In [31]:
columns_tip = list(df_tip.columns)
print(columns_tip)

In [32]:
df_tip.info()

In [33]:
df_tip.describe()

In [39]:
# extrair o horário de funcionamento em colunas separadas (pelo dia da semana)
def extract_hours(hours):
    if pd.isnull(hours):
        return pd.series([None] * 7, index=['hours_Monday', 'hours_Tuesday', 'hours_Wednesday', 'hours_Thursday', 'hours_Friday', 'hours_Saturday', 'hours_Sunday'])
    else:
        return pd.Series([hours.get(day, None) for day in ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']], index=['hours_Monday', 'hours_Tuesday', 'hours_Wednesday', 'hours_Thursday', 'hours_Friday', 'hours_Saturday', 'hours_Sunday'])

df_hours = df_negocios['hours'].apply(extract_hours)
df_negocios = pd.concat([df_negocios, df_hours], axis=1)

# separar atributos em colunas

def extract_atributes(atributes):
    if pd.isnull(atributes):
        return pd.Series()
    else:
        return pd.Series(atributes)
    
df_atributes = df_negocios['attributes'].apply(extract_atributes)
df_negocios = pd.concat([df_negocios, df_atributes], axis=1)

print(df_negocios.head())


KeyError: 'hours'

In [ ]:
columns = list(df_negocios.columns)
print(columns)

In [ ]:
df_negocios.info()

In [ ]:
columns_check = ['attributes', 'categories', 'hours']
df_negocios_cl = df_negocios.dropna(subset=columns_check)
print(df_negocios_cl.info())


In [ ]:
# checar espaços na string
str_cols =  list(df_negocios.select_dtypes(include='object').columns)
str_cols = [col for col in str_cols if col not in ['postal_code', 'latitude', 'longitude', 'stars', 'review_count','is_open', 'hours','attributes']]

for col in str_cols:
    df_negocios[col] = df_negocios[col].str.strip()

print(df_negocios[str_cols].head())

In [ ]:
df_negocios_cl.head()

In [ ]:
#checar latitude e longitude fora do esperado
df_negocios_cl = df_negocios_cl[(df_negocios_cl['latitude'].between(-90, 90)) & (df_negocios_cl['longitude'].between(-180, 180))]

In [ ]:
# checar duplicados
df_negocios_cl = df_negocios_cl.drop_duplicates(subset=['business_id'], keep='first')

In [ ]:
# ver o percentual dos dados excluidos
excluidos = (len(df_negocios) - len(df_negocios_cl)) / len(df_negocios) * 100
print(excluidos)

In [ ]:
# checar dados faltantes
print(df_negocios_cl.isnull().sum())


In [ ]:
df_negocios_cl.head()

In [ ]:
df_negocios_cl.info()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.boxplot(x=df_negocios_cl['review_count'])
plt.show()

sns.histplot(df_negocios_cl['stars'], bins=10, kde=True)
plt.show()


In [24]:
import folium
from folium.plugins import MarkerCluster


# Criar um mapa básico com pontos de latitude/longitude
mapa = folium.Map(location=[df_negocios_cl['latitude'].mean(), df_negocios_cl['longitude'].mean()], zoom_start=10)

marker_cluster = MarkerCluster().add_to(mapa)

for _, row in df_negocios_cl.iterrows():
    folium.Marker(location=[row['latitude'], row['longitude']], popup=row['name']).add_to(marker_cluster)

mapa.save('mapa.html')


In [ ]:
# normalizar textos
df_negocios_cl['categories'] = df_negocios_cl['categories'].str.lower().str.strip()
df_negocios_cl['city'] = df_negocios_cl['city'].str.lower().str.strip()



In [36]:
estado_contagem = df_negocios_cl['state'].value_counts()
print(estado_contagem)

state
PA    26317
FL    20974
TN     9619
IN     8856
MO     8848
AZ     7793
LA     7644
NJ     6620
NV     5844
AB     4111
CA     3951
ID     3600
DE     1761
IL     1669
TX        2
NC        1
WA        1
CO        1
MT        1
MI        1
HI        1
SD        1
VI        1
VT        1
Name: count, dtype: int64
